### 0. 비교분석을 위해 LDA model 만들기.

In [1]:
import pandas as pd
import numpy as np
import pickle
from pprint import pprint
import re

In [2]:
# load data
with open("data/cleaned_katalk_data.pk", "rb") as f:
    data = pickle.load(f)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14911 entries, 0 to 14910
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     14911 non-null  object
 1   user     14911 non-null  object
 2   message  14911 non-null  object
dtypes: object(3)
memory usage: 349.6+ KB
None


### 2. 분석시기 설정하기

In [3]:
# 시간정보 열을 datetime 정보로 변환
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.head()

,user,message
date,,
2016-03-07 15:41:00,YH,안녕하세요 이게 빠르지 않을까 해서요 엠티 관련해서는 회장님께서 이미 밴드에 올려 ...
2016-03-07 15:51:00,WH,네 안녕하세요
2016-03-07 17:14:00,YH,넵 저희 카톡방을 만들어서 거기다 투표를 돌릴까요 아님 그냥 밴드에 두개로 할까요 총무님
2016-03-07 17:21:00,KS,엠티관련해서 참석여부 투표를 말씀하신건가요
2016-03-07 17:22:00,YH,네 저희 이번 토욜 관련해서도 투표를 올려야 해서요


In [4]:
# LDA vs DTM 비교하기 위해 세번째 시간대만 사용

slice_data = data['2016-04-21' : '2016-04-30']
slice2 = list(slice_data["message"])

tokenized_data = [msg.split() for msg in slice2]

print(tokenized_data[:10])
print(len(tokenized_data))

[['오늘아침', '못먹은사람'], ['배달시켜'], ['점심을', '나가서', '먹어야지용', '월급날'], [], [], ['월급날이로구나'], ['생뜡맞게', '주네'], ['우린', '21일'], ['우린', '25'], ['공무원은']]
8279


### 3. LDA & Dynamic Topic Model 돌리기

In [5]:
from gensim.models import ldamodel
from gensim.models import ldaseqmodel
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim import corpora
from tqdm import tqdm_notebook
from time import time

import os

##### dictionary와 doc2bow 만들기 ( LDA에서 2019년도 것만 사용)

In [6]:
dict_path = 'data/kakao_LDA_dict'
corpus_path = 'data/kakao_LDA_corpus'

# Create Dictionary
if not os.path.exists(dict_path):
    dictionary = corpora.Dictionary(tokenized_data)
    # dictionary.filter_extremes(no_below=5, no_above=500)  # 이 줄의 코드는 n회 미만 또는 n회 이상을 지울 때 사용
    dictionary.save(dict_path)
    print(dictionary)
else:
    dictionary = Dictionary.load(dict_path)

# Term Document Frequency (convert tokenized documents into a Document-Term Matrix)    
if not os.path.exists(corpus_path):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize(corpus_path, corpus)
else:
    corpus = bleicorpus.BleiCorpus(corpus_path)

##### Run LDA model 

In [7]:
# DTM 분석에서 best topic으로 나온 결과를 비교하기 위해 같은 토픽 수로 설정.
NUM_TOPICS = 7

model_path = 'data/kakao_LDA_model'

if not os.path.exists(model_path):
    lda_model = ldamodel.LdaModel(corpus=corpus, 
                                  id2word=dictionary, 
                                  num_topics=NUM_TOPICS, 
                                  passes=100)
    lda_model.save(model_path)
else:
    lda_model = ldamodel.LdaModel.load(model_path)

##### Run DTM Model

In [8]:
# DTM 분석에서 best topic으로 나온 결과를 비교하기 위해 같은 토픽 수로 설정.
NUM_TOPICS = 7

dtm_model = ldaseqmodel.LdaSeqModel.load('data/kakao_dtm_model_7')

##### LDA와 DTM 결과 비교해보기

In [9]:
lda_model.show_topic(topicid=0, topn=20)

[('그날', 0.008155815),
 ('그건', 0.007037851),
 ('다시', 0.0050324467),
 ('그런가', 0.0050263726),
 ('그러면', 0.0049440335),
 ('저', 0.004844301),
 ('없어', 0.004814049),
 ('보람이가', 0.0045643146),
 ('뭔가', 0.004166773),
 ('소오름', 0.0039155763),
 ('쉬고', 0.003876249),
 ('재훈오빠가', 0.0037076846),
 ('욜', 0.0035633778),
 ('가자', 0.0035242694),
 ('같은데', 0.003093129),
 ('어디', 0.0030810277),
 ('바로', 0.0028218941),
 ('병준이', 0.00279744),
 ('형이', 0.002796978),
 ('그래도', 0.0027803194)]

In [10]:
dtm_model.print_topic(topic=0, time=2, top_terms=20)

[('나', 0.033816708414789905),
 ('왜', 0.024191943791049563),
 ('진짜', 0.01761018664642402),
 ('응', 0.013864547387638664),
 ('근데', 0.013555867878687303),
 ('좀', 0.013166824883927446),
 ('역시', 0.010397512812373274),
 ('아직', 0.009547127812730783),
 ('넵', 0.008680986684372775),
 ('내가', 0.007755850505890038),
 ('엄청', 0.00683608584465352),
 ('그래도', 0.006532071676062843),
 ('같은데', 0.005113264097751727),
 ('이거', 0.004985935729842827),
 ('유희야', 0.004747337244422429),
 ('둘이', 0.0043832969566334375),
 ('왕왕', 0.004302965551743357),
 ('웅', 0.004185587760810772),
 ('갑자기', 0.0041576647699530424),
 ('유희', 0.004083303032778055)]

##### coherence score 계산 비교

In [11]:
# LDA의 coherence score
lda_cs = CoherenceModel(model=lda_model, 
                        texts=tokenized_data, 
                        corpus=corpus, 
                        dictionary=dictionary,
                        coherence='c_v').get_coherence()

In [12]:
# DTM의 coherence score
dtm_corpus = corpus = bleicorpus.BleiCorpus('data/kakao_DTM_corpus')
dtm_dictionary = Dictionary.load('data/kakao_DTM_dict')
processing_data = [msg.split() for msg in data['message']]

topics_dtm = dtm_model.dtm_coherence(time=2)

dtm_cs = CoherenceModel(topics=topics_dtm, 
                        texts=processing_data, 
                        corpus=dtm_corpus,
                        dictionary=dtm_dictionary, 
                        coherence='c_v').get_coherence()

In [13]:
lda_cs, dtm_cs

(0.763727078961442, 0.7222625292745228)

In [15]:
# coherence score - 각 모델이 데이터들의 확률값에 수렴이 잘 되었는지에 대한 지표
# DTM - 시간의 흐름에 따른 토픽의 변화에 영향을 받는다.